In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from torchvision import transforms

# 1. 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # ResNet 입력 크기에 맞게 확대
    transforms.Grayscale(num_output_channels=3),  # RGB로 변환
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # ImageNet 평균
                         [0.229, 0.224, 0.225])   # ImageNet 표준편차
])

# 2. 커스텀 Dataset 정의
class DigitsDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.images[idx]
        img = Image.fromarray((img * 16).astype(np.uint8))  # [0-1] → [0-255]로 확장
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return img, label

# 3. 데이터 로드 및 분할
digits = load_digits()
X = digits.images / 16.0  # 정규화 [0,1]
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = DigitsDataset(X_train, y_train, transform=transform)
test_dataset = DigitsDataset(X_test, y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 4. 사전학습된 ResNet 모델
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)  # 0~9 숫자 분류

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 5. 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 6. 학습 함수
def train(model, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"[{epoch+1}/{epochs}] Loss: {running_loss / len(train_loader):.4f}")

# 학습 실행
train(model, epochs=5)


[1/5] Loss: 0.7998
[2/5] Loss: 0.0715
[3/5] Loss: 0.0219
[4/5] Loss: 0.0116
[5/5] Loss: 0.0071


In [3]:
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import torch

# 평가
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # 예측
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1) 

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())


# 평가 지표 출력
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds,  average='macro')
f1 = f1_score(all_labels,all_preds, average='macro')
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f"Accuracy     : {accuracy:.4f}")
print(f"Precision    : {precision:.4f}")
print(f"Recall       : {recall:.4f}")
print(f"F1 Score     : {f1:.4f}")

Accuracy     : 0.9972
Precision    : 0.9966
Recall       : 0.9967
F1 Score     : 0.9966
